In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

def calculate_coverage(selected_facilities, demand_radius, locations):
    covered_indices = set()
    for facility_index in selected_facilities:
        facility_location = locations[facility_index]
        within_coverage = np.linalg.norm(locations - facility_location, axis=1) <= demand_radius
        covered_indices.update(np.where(within_coverage)[0])
    return covered_indices

# Google 드라이브 마운트
drive.mount('/content/drive')

# 파일 경로 설정
path_to_files = '/content/drive/My Drive/mclp/'
numdata_file = path_to_files + 'numdata.csv'

# numdata.csv 파일 로드
merged_df = pd.read_csv(numdata_file)

# 데이터 프레임에서 위치 정보 추출 (위도, 경도)
locations = merged_df[['위도', '경도']].values

# MCLP 알고리즘을 위한 데이터 변환
data = merged_df[['수요 지수', '인프라 지수', '접근성 지수', '경도']].values

# Combine all three criteria into a single weight
weights = 0.6 * data[:, 0] + 0.2 * data[:, 1] + 0.2 * data[:, 2]

# MCLP 알고리즘 적용
num_facilities = 10  # 선택할 시설의 수
demand_radius = 0.0063  # Covering radius for demand points

selected_facilities = []
remaining_indices = set(range(len(data)))
facility_scores = []

for _ in range(num_facilities):
    total_weights = np.sum(weights[list(remaining_indices)])
    covered_indices = calculate_coverage(selected_facilities, demand_radius, locations)
    uncovered_indices = remaining_indices - covered_indices

    if not uncovered_indices:
        break

    selected_index = max(uncovered_indices, key=lambda i: weights[i])
    selected_facilities.append(selected_index)
    remaining_indices.remove(selected_index)
    facility_scores.append(weights[selected_index])

# 선택된 시설의 점수에 따라 정렬
sorted_facilities = [x for _, x in sorted(zip(facility_scores, selected_facilities), reverse=True)]

# 결과를 데이터프레임으로 변환
result_df = merged_df.iloc[sorted_facilities]

# 결과를 CSV 파일로 저장
result_path = path_to_files + 'selected_facilities.csv'
result_df.to_csv(result_path, index=False)

# 결과 출력
print("Selected Facility IDs in order:", sorted_facilities)
